In [154]:
import numpy as np
import pandas as pd
from importlib import reload

import munch
import datetime

import shared
import config
import provider_yfinance as provider

reload(shared)
reload(config)
reload(provider)

cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
config.overwrite_end_dt(cfg, '2019-12-19')
config.save_config(cfg)

config> read from config file: 'Munch({'base': Munch({'cache_dir': '/mnt/c/notebooks/sandbox/cache', 'cache_enabled': False, 'config_file_path': '/mnt/c/notebooks/sandbox/config.json', 'current_dir': '/mnt/c/notebooks/sandbox', 'model_dir': '/mnt/c/notebooks/sandbox/model'}), 'datasets': Munch({'raw': Munch({'benchmarks': ['000001.SS', 'AUDUSD=X', 'BTCUSD=X', 'CNY=X', 'ETHUSD=X', 'EURCAD=X', 'EURCHF=X', 'EURGBP=X', 'EURHUF=X', 'EURJPY=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBPJPY=X', 'GBPUSD=X', 'GC=F', 'HKD=X', 'IDR=X', 'IMOEX.ME', 'INR=X', 'JPY=X', 'KW=F', 'LC=F', 'LH=F', 'MXN=X', 'MYR=X', 'NZDUSD=X', 'PA=F', 'PHP=X', 'PL=F', 'RUB=X', 'S=F', 'SGD=X', 'SI=F', 'SM=F', 'THB=X', 'ZAR=X', '^AORD', '^AXJO', '^BFX', '^BSESN', '^BUK100P', '^BVSP', '^DJI', '^FCHI', '^FTSE', '^GDAXI', '^GSPC', '^GSPTSE', '^HSI', '^IPSA', '^IXIC', '^JKSE', '^JN0U.JO', '^KLSE', '^KS11', '^MERV', '^MXX', '^N100', '^N225', '^NYA', '^NZ50', '^RUT', '^STI', '^STOXX50E', '^TA125.TA', '^TWII', '^VIX', '^XAX'], 'stoc

In [2]:
%%time 
# 344 ms
cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

CPU times: user 219 ms, sys: 93.8 ms, total: 312 ms
Wall time: 341 ms


In [3]:
data_benchmarks.tickers['^GDAXI'].history.head()

,open,high,low,close,volume,dividends,stock_splits
date,,,,,,,
1987-12-30,1005.19,1005.19,1005.19,1005.19,0,0,0
1988-01-04,956.49,956.49,956.49,956.49,0,0,0
1988-01-05,996.10,996.10,996.10,996.10,0,0,0
1988-01-06,1006.01,1006.01,1006.01,1006.01,0,0,0
1988-01-07,1014.47,1014.47,1014.47,1014.47,0,0,0


In [4]:
%%time 
# total: 2.55 s
prep_stocks = provider.prepare_stocks(cfg, data_stocks)
prep_benchmarks = provider.prepare_benchmarks(cfg, data_benchmarks)

CPU times: user 141 ms, sys: 46.9 ms, total: 188 ms
Wall time: 193 ms


In [5]:
prep_benchmarks['^GDAXI'].head()

,open,high,low,close,volume,diff_prev,diff_oc,diff_hl,rolling_3d,rolling_5d,rolling_21d,rolling_35d,rolling_50d,lag_1d,lag_2d,lag_3d,lag_4d,lag_5d
date,,,,,,,,,,,,,,,,,,
2018-02-05,12687.79,12752.60,12622.44,12687.49,113937900,NaN,-0.30,130.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-06,12232.86,12550.16,12232.86,12392.66,204856800,-454.63,159.80,317.30,NaN,NaN,NaN,NaN,NaN,-0.023512,NaN,NaN,NaN,NaN
2018-02-07,12478.68,12651.28,12414.84,12590.43,153471100,86.02,111.75,236.44,12556.86,NaN,NaN,NaN,NaN,0.015833,-0.007679,NaN,NaN,NaN
2018-02-08,12506.18,12541.32,12187.45,12260.29,153991900,-84.25,-245.89,353.87,12414.46,NaN,NaN,NaN,NaN,-0.026571,-0.010739,-0.034251,NaN,NaN
2018-02-09,12263.10,12296.18,12003.36,12107.48,175812900,2.81,-155.62,292.82,12319.40,12407.67,NaN,NaN,NaN,-0.012542,-0.039114,-0.023281,-0.046793,NaN


In [6]:
%%time
# total: 1.2 s
enc_stocks = provider.encode_stocks(cfg, prep_stocks)
enc_benchmarks = provider.encode_benchmarks(cfg, prep_benchmarks, prep_stocks)

CPU times: user 31.2 ms, sys: 78.1 ms, total: 109 ms
Wall time: 96.5 ms


In [7]:
enc_benchmarks['^GDAXI'].head()

,open,high,low,close,volume,diff_prev,diff_oc,diff_hl,rolling_3d,rolling_5d,rolling_21d,rolling_35d,rolling_50d,lag_1d,lag_2d,lag_3d,lag_4d,lag_5d,scaled_volume
date,,,,,,,,,,,,,,,,,,,
2018-02-05,12687.79,12752.60,12622.44,12687.49,113937900.0,-454.63,-0.30,130.16,12556.86,12407.67,12361.606667,12316.783143,12283.5436,-0.023512,-0.007679,-0.034251,-0.046793,-0.032419,0.284727
2018-02-06,12232.86,12550.16,12232.86,12392.66,204856800.0,-454.63,159.80,317.30,12556.86,12407.67,12361.606667,12316.783143,12283.5436,-0.023512,-0.007679,-0.034251,-0.046793,-0.032419,0.511930
2018-02-07,12478.68,12651.28,12414.84,12590.43,153471100.0,86.02,111.75,236.44,12556.86,12407.67,12361.606667,12316.783143,12283.5436,0.015833,-0.007679,-0.034251,-0.046793,-0.032419,0.383519
2018-02-08,12506.18,12541.32,12187.45,12260.29,153991900.0,-84.25,-245.89,353.87,12414.46,12407.67,12361.606667,12316.783143,12283.5436,-0.026571,-0.010739,-0.034251,-0.046793,-0.032419,0.384821
2018-02-09,12263.10,12296.18,12003.36,12107.48,175812900.0,2.81,-155.62,292.82,12319.40,12407.67,12361.606667,12316.783143,12283.5436,-0.012542,-0.039114,-0.023281,-0.046793,-0.032419,0.439351


In [155]:
%%time 
# total: 18min - 25min

for submodel_settings in cfg.train.settings:
    print(f"sm-{submodel_settings.id}> preparing submodel data ...")
    model_data = provider.prepare_submodel_data(cfg, submodel_settings, enc_stocks, enc_benchmarks)
    # update num_features setting (informational)
    submodel_settings.num_features = len(model_data.X[0][0][0][0])
config.save_config(cfg)

sm-nr_1-lookback_3-label_1> preparing submodel data ...
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20191219/nr_1-lookback_3-label_1/submodel_data.pkl'
sm-nr_1-lookback_3-label_1> generating relative benchmarks data ...
sm-nr_1-lookback_3-label_1> preparing stock: '1COV.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'ADS.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'ALV.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'BAS.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'BAYN.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'BEI.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'BMW.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'CON.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'DAI.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'DB1.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'DBK.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'DPW.DE' ...
sm-nr_1-lookback_3-label_1> preparing stock: 'DTE.DE' ...


In [235]:
# df = model_data
df = model_data[model_data.ticker=='SAP.DE']
num_samples = df.shape[0]
input_length = submodel_settings.lookback_days
input_dim = submodel_settings.num_features
output_dim = 1

X_train = np.hstack(np.asarray(df.X)).reshape(num_samples, input_length, input_dim)
y_train = np.hstack(np.asarray(df.y)).reshape(num_samples, output_dim)

In [239]:
input_dim//16

82

In [253]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Masking

model = Sequential()
model.add(BatchNormalization(input_shape=(input_length, input_dim)))
model.add(Masking())
model.add(LSTM(input_dim, dropout=.3, return_sequences=True, activation="softsign"))
model.add(LSTM(input_dim//2, dropout=.3, return_sequences=True, activation="softsign"))
model.add(LSTM(input_dim//2, dropout=.3, activation="softsign"))
model.add(Dense(output_dim))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy', 'mean_squared_error'])

# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=0, mode='auto')
# checkpointer = ModelCheckpoint(filepath=filepath + "best_weights.hdf5", verbose=0, save_best_only=True)
# history = model.fit(X_train, y_train, batch_size=200, nb_epoch=3, verbose=1, callbacks=[monitor, checkpointer])
history = model.fit(X_train, y_train, batch_size=200, epochs=10, verbose=1)

# single stock
# all stocks

Epoch 1/10
161/161 [==============================] - 34s 212ms/step - loss: 0.7645 - acc: 0.0000e+00 - mean_squared_error: 0.7645
Epoch 2/10
161/161 [==============================] - 2s 10ms/step - loss: 0.5623 - acc: 0.0000e+00 - mean_squared_error: 0.5623
Epoch 3/10
161/161 [==============================] - 1s 9ms/step - loss: 0.3010 - acc: 0.0000e+00 - mean_squared_error: 0.3010
Epoch 4/10
161/161 [==============================] - 1s 7ms/step - loss: 0.2498 - acc: 0.0000e+00 - mean_squared_error: 0.2498
Epoch 5/10
161/161 [==============================] - 1s 9ms/step - loss: 0.2045 - acc: 0.0000e+00 - mean_squared_error: 0.2045
Epoch 6/10
161/161 [==============================] - 1s 8ms/step - loss: 0.1042 - acc: 0.0000e+00 - mean_squared_error: 0.1042
Epoch 7/10
161/161 [==============================] - 1s 9ms/step - loss: 0.1188 - acc: 0.0000e+00 - mean_squared_error: 0.1188
Epoch 8/10
161/161 [==============================] - 1s 8ms/step - loss: 0.0836 - acc: 0.0000e+00 -

In [234]:
df_test = pd.DataFrame({
    'X': pd.Series(model.predict(X_train).flatten()),
    'y': y_train.flatten()
})
df_test['diff'] = df_test.X - df_test.y
df_test

,X,y,diff
0,-0.109018,-0.053866,-0.055152
1,0.295605,0.326008,-0.030404
2,1.071603,3.367844,-2.296242
3,0.952732,-0.370991,1.323723
4,-0.424404,-1.489826,1.065421
...,...,...,...
4792,0.285366,-0.576471,0.861837
4793,0.285366,-0.576471,0.861837
4794,0.285366,-0.576471,0.861837
4795,0.285366,-0.576471,0.861837


In [ ]:
sequences = []
max_samples = cfg.model.max_samples
for i in range(max_seq_len):
    seq = []
    sequences += [seq]
pad_sequences([sequences], maxlen=max_seq_len, dtype='float32')


In [ ]:
https://stackoverflow.com/questions/39674713/neural-network-lstm-input-shape-from-dataframe
https://stackoverflow.com/questions/49803503/lstm-preprocessing-build-3d-arrays-from-pandas-data-frame-based-on-id

In [ ]:
df = prep_stocks['1COV.DE']
for k in prep_benchmarks.keys():
    df = pd.merge(df, prep_benchmarks[k], how='left', left_index=True, right_index=True)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)


            # prefix = f'{to_snake_case(k)}_'
            # df.rename(columns=dict([(c, f'{prefix}{c}') for c in df.columns]), inplace=True)       
df

In [ ]:
prep_benchmarks['000001.SS']